# Mean-Variance Optimization

In [1]:
import sys

from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns

In [2]:
# 경로설정

CWD = Path.cwd()
WORKSPACE_PATH = CWD.parent
COMMON_PATH = WORKSPACE_PATH / 'common'
DATA_PATH = WORKSPACE_PATH / 'data'
FONT_PATH = COMMON_PATH / 'fonts' / 'NanumGothic.ttf'

In [3]:
font_prop = font_manager.FontProperties(fname=FONT_PATH)
rc('font', family=font_prop.get_name())

In [4]:
sys.path.append(str(COMMON_PATH))

In [5]:
import fn_loader as fn
import fn_config as fncfg

from validator import Alpha

## Import Data

In [6]:
# loader 생성 (데이터 로드 + multi index로 전처리)
# use_alias=True 일 경우 긴 ETF명 대신 축약한 단어 사용
pricevolume1 = fn.FnDataLoader('DBGAPS2024_pricevolume.xlsx', use_alias=True)  

In [7]:
pricevolume1.get_datafields()

Index(['return', 'adjopen', 'adjhigh', 'adjlow', 'adjclose', 'moneyvolume'], dtype='object')

In [8]:
# pricevolume2 = fn.FnDataLoader('DBGAPS2024_pricevolume2.xlsx', use_alias=True)

In [9]:
# pricevolume2.get_datafields()

In [10]:
return_df = pricevolume1.get_data('return') # 
return_df.head() 

,shortterm,usd,usdinv,kodexinv,oil,gold,hybond,midbond,10y,csi300,nikkei,euro50,sp500,kosdaq150,kodex200
Date,,,,,,,,,,,,,,,
2016-03-04,0.00,-0.96,0.92,-0.12,1.27,1.83,0.37,-0.20,-0.19,0.42,0.19,-0.46,0.43,0.28,0.02
2016-03-07,0.00,-0.36,0.19,-0.24,4.58,-0.21,0.37,0.17,0.00,0.48,-0.24,-0.05,0.06,-0.51,0.21
2016-03-08,-0.01,0.45,-0.43,0.79,0.93,0.67,0.05,0.00,0.29,-0.89,-0.48,-0.05,-0.11,0.09,-0.70
2016-03-09,0.02,0.97,-0.82,-0.42,-1.45,-1.07,-0.05,-0.14,-0.05,-0.35,-1.00,-0.57,-0.21,0.60,0.43
2016-03-10,-0.01,-1.08,1.12,-0.73,1.74,-0.46,-0.10,0.19,-0.10,0.21,1.16,0.10,0.17,0.55,0.92


## MVO

In [11]:
# Parameters

WINDOW = 20

In [12]:
# Function to compute the tangent portfolio weights using closed-form solution
def tangent_portfolio_weights(mean_returns, cov_matrix):
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    ones = np.ones(len(mean_returns))
    tangent_weights = np.dot(inv_cov_matrix, mean_returns)
    tangent_weights /= np.dot(ones, tangent_weights)
    return tangent_weights


In [13]:
# List to store the results and their corresponding dates
tangent_portfolios = []
rebalance_dates = []

# Loop through the dataset with the specified window
for start in range(0, len(return_df) - WINDOW + 1, WINDOW):
    end = start + WINDOW
    window_data = return_df[start:end]
    
    # Calculate the mean returns and covariance matrix for the window
    mean_returns = window_data.mean().values
    cov_matrix = window_data.cov().values
    
    # Compute the tangent portfolio weights using the closed-form solution
    tangent_weights = tangent_portfolio_weights(mean_returns, cov_matrix)
    
    # Store the tangent portfolio weights and the corresponding rebalance date
    tangent_portfolios.append(tangent_weights)
    rebalance_dates.append(return_df.index[end-1])

# Convert the list of tangent portfolios to a DataFrame
tangent_portfolios_df = pd.DataFrame(tangent_portfolios, index=rebalance_dates, columns=return_df.columns)

# Display the resulting DataFrame of tangent portfolio weights
tangent_portfolios_df.head()


,shortterm,usd,usdinv,kodexinv,oil,gold,hybond,midbond,10y,csi300,nikkei,euro50,sp500,kosdaq150,kodex200
2016-03-31,0.948809,0.011406,0.018267,-0.011582,0.005114,-0.015002,0.015295,-0.018544,0.041146,-0.016203,-0.000289,-0.009118,0.046988,-0.011598,-0.004691
2016-04-29,0.915505,0.032745,0.028287,0.069302,0.006757,-0.025602,0.059729,-0.279076,0.154339,-0.012394,-0.010097,0.004433,0.004437,-0.005199,0.056833
2016-05-31,1.119990,-0.101811,-0.121403,0.012105,0.004959,-0.000730,0.025001,-0.086753,0.125693,0.008237,-0.004628,0.020287,0.011496,-0.006049,-0.006394
2016-06-29,0.882970,-0.036727,0.002915,-0.063515,-0.003138,-0.009640,-0.022565,0.430402,-0.080325,-0.002859,0.009702,0.000204,-0.049995,-0.004591,-0.052838
2016-07-27,0.990045,0.002476,0.009108,-0.010704,-0.003030,0.000531,0.001464,0.054926,-0.035835,0.005098,-0.002267,0.000779,0.003862,-0.005153,-0.011299


In [14]:
np.isclose(tangent_portfolios_df.sum(axis=1), 1.0).all()

True

In [15]:
weight_df = tangent_portfolios_df.reindex(return_df.index, method='ffill')
weight_df.dropna(inplace=True)
weight_df.tail()

,shortterm,usd,usdinv,kodexinv,oil,gold,hybond,midbond,10y,csi300,nikkei,euro50,sp500,kosdaq150,kodex200
Date,,,,,,,,,,,,,,,
2024-05-10,0.119337,-0.008678,-0.010479,0.159983,-0.003417,0.005423,0.038115,0.678163,-0.103144,-0.006857,0.003574,0.008332,-0.027496,-0.002846,0.14999
2024-05-13,0.119337,-0.008678,-0.010479,0.159983,-0.003417,0.005423,0.038115,0.678163,-0.103144,-0.006857,0.003574,0.008332,-0.027496,-0.002846,0.14999
2024-05-14,0.119337,-0.008678,-0.010479,0.159983,-0.003417,0.005423,0.038115,0.678163,-0.103144,-0.006857,0.003574,0.008332,-0.027496,-0.002846,0.14999
2024-05-16,0.119337,-0.008678,-0.010479,0.159983,-0.003417,0.005423,0.038115,0.678163,-0.103144,-0.006857,0.003574,0.008332,-0.027496,-0.002846,0.14999
2024-05-17,0.119337,-0.008678,-0.010479,0.159983,-0.003417,0.005423,0.038115,0.678163,-0.103144,-0.006857,0.003574,0.008332,-0.027496,-0.002846,0.14999


## Alpha validation

In [16]:
mvo = Alpha('jaepil', weight_df)

AssertionError: Individual asset weight constraints violated